In [11]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Para tratamiento de fechas
import datetime as dt
from datetime import timedelta

In [12]:
os.chdir('E:/DS/Kaggle')

In [3]:
# Load data
pageviews = pd.read_csv('./pageviews.csv')
# content_category = pd.read_csv('./CONTENT_CATEGORY.csv')
# content_category_bottom = pd.read_csv('./CONTENT_CATEGORY_BOTTOM.csv')
# content_category_top = pd.read_csv('./CONTENT_CATEGORY_TOP.csv')
device_data = pd.read_csv('./device_data.csv')
# page = pd.read_csv('./PAGE.csv')
# site_id = pd.read_csv('./SITE_ID.csv')
conversiones = pd.read_csv('./conversiones.csv', dtype={'mes' : 'object', 'anio' : 'object', 'USER_ID' : 'object'})

###  Procesamiento archivo conversiones

Elimino el '.0' de los datos de conversiones

In [4]:
conversiones['mes'] = pd.to_numeric(conversiones['mes'].str.replace('\.0', ''))
conversiones['anio'] = pd.to_numeric(conversiones['anio'].str.replace('\.0', ''))
conversiones['USER_ID'] = pd.to_numeric(conversiones['USER_ID'].str.replace('\.0', ''))
conversiones['conversion'] = 1
conversiones.head()

,mes,anio,USER_ID,conversion
0,7,2018,1410,1
1,8,2018,10755,1
2,8,2018,8270,1
3,10,2018,7558,1
4,9,2018,10731,1


### Procesamiento archivo pageviews

Crear columnas anio y mes

In [6]:
pageviews['USER_ID'] = pd.to_numeric(pageviews['USER_ID'])

# Creo campos para hacer el join con el archivo conversiones
pageviews['FEC_EVENT'] = pd.to_datetime(pageviews['FEC_EVENT'])
pageviews['anio'] = pageviews['FEC_EVENT'].dt.year
pageviews['mes'] = pageviews['FEC_EVENT'].dt.month
pageviews.head()

,FEC_EVENT,PAGE,CONTENT_CATEGORY,CONTENT_CATEGORY_TOP,CONTENT_CATEGORY_BOTTOM,SITE_ID,ON_SITE_SEARCH_TERM,USER_ID,anio,mes
0,2018-03-30 07:35:48,1,1,1,1,1,1,0,2018,3
1,2018-03-30 07:35:52,2,2,2,2,2,1,0,2018,3
2,2018-03-30 07:36:11,3,2,2,2,3,1,0,2018,3
3,2018-03-30 07:36:16,4,2,2,2,3,1,0,2018,3
4,2018-03-30 07:41:38,5,2,2,2,2,1,0,2018,3


### Merge datasets pageviews y conversiones
Hago un join y lleno con 0 en los casos que no hubo conversion

In [7]:
data = pd.merge(pageviews, conversiones, how='left')
data['conversion'] = data['conversion'].replace(np.nan, 0)
data.head()

,FEC_EVENT,PAGE,CONTENT_CATEGORY,CONTENT_CATEGORY_TOP,CONTENT_CATEGORY_BOTTOM,SITE_ID,ON_SITE_SEARCH_TERM,USER_ID,anio,mes,conversion
0,2018-03-30 07:35:48,1,1,1,1,1,1,0,2018,3,0.0
1,2018-03-30 07:35:52,2,2,2,2,2,1,0,2018,3,0.0
2,2018-03-30 07:36:11,3,2,2,2,3,1,0,2018,3,0.0
3,2018-03-30 07:36:16,4,2,2,2,3,1,0,2018,3,0.0
4,2018-03-30 07:41:38,5,2,2,2,2,1,0,2018,3,0.0


In [8]:
print('Usuarios con conversión en el último cuarto:')
print(data.loc[(data['mes'] >= 10) & (data['conversion'] == 1.0),:]['USER_ID'].nunique())
print('Usuarios sin conversión en el último cuarto:')
print(data.loc[(data['mes'] >= 10) & (data['conversion'] != 1.0),:]['USER_ID'].nunique())

Usuarios con conversión en el último cuarto:
309
Usuarios sin conversión en el último cuarto:
11343


### Sesiones por usuario

In [9]:
# Definimos un límite de tiempo entre eventos (Cuánto tiempo pasa hasta que expira la sesión?)
T = timedelta(seconds=30*60)

# data = data.loc[:,['USER_ID', 'FEC_EVENT', 'anio', 'mes']]
# Ordenamos el dataset por usuario y fecha de evento
data = data.sort_values(by=['USER_ID','FEC_EVENT'])

# Agregamos una columna con la fecha de evento anterior
data['last_date'] = data.groupby(['USER_ID'])['FEC_EVENT'].shift()

# Crear flag de nueva sesión cada vez que hay un nuevo ingreso
data['new_session'] = (((data['FEC_EVENT']
                            - data['last_date'])>=T) | (data['last_date'].isnull())).astype(int)
data['inc'] = data.groupby('USER_ID')['new_session'].cumsum()

# Creamos el id de sesión
data['session_id'] = data['USER_ID'].astype(str) + '_' + data['inc'].astype(str)
data = data.drop('inc', axis=1)

print(data.head())

              FEC_EVENT  PAGE  CONTENT_CATEGORY  CONTENT_CATEGORY_TOP  \
284 2018-01-01 09:56:47    14                 4                     2   
285 2018-01-01 09:57:19    14                 4                     2   
286 2018-01-01 09:57:48    12                 4                     2   
287 2018-01-01 09:57:48    10                 4                     2   
288 2018-01-01 09:57:49    11                 4                     2   

     CONTENT_CATEGORY_BOTTOM  SITE_ID  ON_SITE_SEARCH_TERM  USER_ID  anio  \
284                        4        2                    1        0  2018   
285                        4        2                    1        0  2018   
286                        4        2                    1        0  2018   
287                        4        2                    1        0  2018   
288                        4        2                    1        0  2018   

     mes  conversion           last_date  new_session session_id  
284    1         0.0           

# Agrupar por id de usuario, año y mes para el dataset final

Para cada usuario, año y mes, tengo la cantidad de sesiones, si convirtió o no en ese mes, y la fecha del último evento

In [11]:
grouped_data = data.groupby(['USER_ID','anio','mes'], as_index=False).agg({'new_session':'count', 'conversion':'sum', 'FEC_EVENT':'max'})
grouped_data['conversion'] = np.where(grouped_data['conversion']> 0, 1, 0)

grouped_data.shape

(104255, 6)

In [17]:
# Load data
grouped_data = pd.read_csv('./grouped_data.csv')
grouped_data['FEC_EVENT'] = pd.to_datetime(grouped_data['FEC_EVENT'])

In [18]:
grouped_data.head()

,Unnamed: 0,USER_ID,anio,mes,PAGE,conversion+0,FEC_EVENT,gp_prob,dm_prob,ds_prob,h_prob
0,0,0,2018,1,710,0,2018-01-31 12:16:28,0.045478,0.042184,0.040192,0.040449
1,1,0,2018,2,375,0,2018-02-28 21:38:43,0.045314,0.039823,0.040293,0.041754
2,2,0,2018,3,756,0,2018-03-31 20:08:59,0.043872,0.040972,0.040376,0.040694
3,3,0,2018,4,497,0,2018-04-30 08:29:50,0.044660,0.040243,0.040716,0.040218
4,4,0,2018,5,496,0,2018-05-31 13:58:21,0.044371,0.041287,0.040377,0.040260


## Dataset para entrenar el modelo
Entrenamos con los datos de los meses 1 a 9 de 2018, para luego predecir en el último trimestre del 2018

Features:
- promedio de sesiones por mes
- cantidad de conversiones anteriores
- tiempo desde el último evento

In [19]:
byuser = grouped_data.loc[grouped_data['mes'] < 10,]
byuser = byuser.groupby('USER_ID', as_index=False).agg({'PAGE':'count', 'conversion+0':'sum'
                                                   , 'FEC_EVENT':'max', 'gp_prob':'mean'
                                                   ,'dm_prob':'mean','ds_prob':'mean'
                                                   ,'h_prob':'mean'})

# .agg({'new_session':'mean', 'conversion':'sum', 'FEC_EVENT':'max'})
byuser = byuser.rename(columns = {'conversion+0':'prev_conversions'#,'new_session':'session_count'
                                  , 'FEC_EVENT':'last_event'})
byuser['time_since_last_event'] = ((dt.datetime.now() - byuser['last_event'])).astype('timedelta64[s]')
byuser = byuser.drop('last_event', axis = 1)

In [20]:
byuser.head()

,USER_ID,PAGE,prev_conversions,last_event,gp_prob,dm_prob,ds_prob,h_prob
0,0,9,0,2018-09-30 20:13:42,0.044088,0.040570,0.040468,0.040412
1,1,9,0,2018-09-30 20:56:50,0.045900,0.040684,0.040247,0.040735
2,2,9,0,2018-09-28 00:04:58,0.039776,0.040964,0.040324,0.039119
3,3,9,0,2018-09-28 16:03:37,0.038681,0.040411,0.040704,0.040406
4,4,9,0,2018-09-27 22:15:33,0.043370,0.040190,0.040396,0.041350


#### Agregamos los labels

In [23]:
# Convirtieron en el último cuarto?
labels = grouped_data.loc[grouped_data['mes'] >= 10,]
labels = labels.groupby('USER_ID', as_index=False).agg({'conversion+0':'sum'})
labels['conversion+0'] = np.where(labels['conversion+0']> 0, 1, 0)

byuser = pd.merge(byuser, labels, how='inner')
byuser = byuser.set_index('USER_ID', drop=True)
byuser.head()

,PAGE,prev_conversions,gp_prob,dm_prob,ds_prob,h_prob,time_since_last_event,conversion+0
USER_ID,,,,,,,,
0,9,0,0.044088,0.040570,0.040468,0.040412,27487248.0,0
1,9,0,0.045900,0.040684,0.040247,0.040735,27484660.0,0
2,9,0,0.039776,0.040964,0.040324,0.039119,27732572.0,0
3,9,0,0.038681,0.040411,0.040704,0.040406,27675053.0,0
4,9,0,0.043370,0.040190,0.040396,0.041350,27739137.0,0


In [24]:
byuser.to_csv('train.csv', sep=',', header=True)

## Dataset para predecir
Predecimos con los datos de todo 2018 si va a haber una conversión en el primer trimestre del 2019

In [25]:
final = grouped_data.groupby('USER_ID', as_index=False).agg({'PAGE':'count', 'conversion+0':'sum'
                                                   , 'FEC_EVENT':'max', 'gp_prob':'mean'
                                                   ,'dm_prob':'mean','ds_prob':'mean'
                                                   ,'h_prob':'mean'})
#.agg({'new_session':'sum', 'conversion':'sum', 'FEC_EVENT':'max'})
final = final.rename(columns = {'conversion+0':'prev_conversions'#,'new_session':'session_count'
                                  , 'FEC_EVENT':'last_event'})
final['time_since_last_event'] = ((dt.datetime.now() - final['last_event'])).astype('timedelta64[s]')
final = final.drop('last_event', axis = 1)
final = final.set_index('USER_ID', drop=True)
final.head()

,PAGE,prev_conversions,gp_prob,dm_prob,ds_prob,h_prob,time_since_last_event
USER_ID,,,,,,,
0,12,0,0.042834,0.040407,0.040503,0.040402,19575628.0
1,12,0,0.044092,0.040352,0.040239,0.040404,19613227.0
2,12,0,0.039728,0.040621,0.040296,0.038879,19888824.0
3,12,0,0.038693,0.040602,0.040749,0.040377,19647997.0
4,12,0,0.042443,0.040108,0.040391,0.041110,19780455.0


In [26]:
final.to_csv('test.csv', sep=',', header=True)